In [2]:
import numpy

In [2]:
!ls /kaggle/input/mri-images/

13.dcm	14.dcm	1.dcm


In [3]:
!ls /kaggle/input/

first-model  nonmri-images					 second-model
mri-images   rsna-2024-lumbar-spine-degenerative-classification  third-model


In [3]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomResNet50(nn.Module):
    def __init__(self, num_classes=3, pretrained=False):
        super(CustomResNet50, self).__init__()
        weights = ResNet50_Weights.IMAGENET1K_V1 if pretrained else None
        self.model = resnet50(weights=weights)
        
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        self.dropout = nn.Dropout(0.5)  

    def forward(self, x):
        return self.model(x)

    def unfreeze_model(self):
        for param in self.model.parameters():
            param.requires_grad = True

    def unfreeze_specific_layers(self, layer_names=None):
        for name, param in self.model.named_parameters():
            if layer_names is None or any(layer in name for layer in layer_names):
                param.requires_grad = True
            else:
                param.requires_grad = False


sagittal_t1_model = CustomResNet50(num_classes=3).to(device)
axial_t2_model = CustomResNet50(num_classes=3).to(device)
sagittal_t2stir_model = CustomResNet50(num_classes=3).to(device)

for model in [sagittal_t1_model, axial_t2_model, sagittal_t2stir_model]:
    model.unfreeze_model()

weights = torch.tensor([1.0, 2.0, 4.0])
criterion = nn.CrossEntropyLoss(weight=weights.to(device))

optimizer_sagittal_t1 = torch.optim.Adam(sagittal_t1_model.model.fc.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_axial_t2 = torch.optim.Adam(axial_t2_model.model.fc.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_sagittal_t2stir = torch.optim.Adam(sagittal_t2stir_model.model.fc.parameters(), lr=0.001, weight_decay=1e-4)

models = {
    'Sagittal T1': sagittal_t1_model,
    'Axial T2': axial_t2_model,
    'Sagittal T2/STIR': sagittal_t2stir_model,
}

optimizers = {
    'Sagittal T1': optimizer_sagittal_t1,
    'Axial T2': optimizer_axial_t2,
    'Sagittal T2/STIR': optimizer_sagittal_t2stir,
}

for model_name, model in models.items():
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable parameters for {model_name}: {trainable_params}")


Trainable parameters for Sagittal T1: 23514179
Trainable parameters for Axial T2: 23514179
Trainable parameters for Sagittal T2/STIR: 23514179


In [5]:
from torchvision import transforms
import pydicom

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
])

def preprocess_dcm_image(dcm_path):
    dcm = pydicom.dcmread(dcm_path)
    pixel_array = dcm.pixel_array.astype(np.float32)

    pixel_array -= np.min(pixel_array)
    pixel_array /= np.max(pixel_array)
    pixel_array *= 255.0
    pixel_array = pixel_array.astype(np.uint8)

    if len(pixel_array.shape) == 2:
        pixel_array = np.stack([pixel_array]*3, axis=-1)  # (H, W, 3)

    image = Image.fromarray(pixel_array)
    image = transform(image).unsqueeze(0).to(device)  # [1, 3, 224, 224]
    return image


In [6]:
from PIL import Image
import numpy as np

def is_possibly_mri(image_path):
    image = Image.open(image_path).convert("L")
    np_img = np.array(image)

    std_intensity = np.std(np_img)
    unique_vals = np.unique(np_img).size

    if std_intensity < 10 or unique_vals < 50:
        return False 
    if std_intensity > 60 and unique_vals > 200:
        return False 

    return True

In [7]:
class_names = {0: "Normal", 1: "Mild", 2: "Severe"}

In [8]:
from PIL import Image
import torchvision.transforms as transforms

def preprocess_jpg_image(jpg_path):
    image = Image.open(jpg_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ])
    return transform(image).unsqueeze(0).to(device) 


In [9]:
import torch.nn.functional as F

def predict_with_model(image_path, model_name):
    model_paths = {
        "Sagittal T1": "/kaggle/input/first-model/pytorch/default/1/best_model_1.pth",
        "Axial T2": "/kaggle/input/second-model/pytorch/default/1/best_model_4.pth",
        "Sagittal T2/STIR": "/kaggle/input/third-model/pytorch/default/1/best_model_1 (1).pth",
    }

    model = CustomResNet50(num_classes=3).to(device)
    state_dict = torch.load(model_paths[model_name], map_location=device, weights_only=True)
    model.load_state_dict(state_dict)
    model.eval()

    try:
        if image_path.lower().endswith(".dcm"):
            image_tensor = preprocess_dcm_image(image_path)
        else:
            image_tensor = preprocess_jpg_image(image_path)
    except Exception as e:
        print(f"Failed to read or preprocess the image: {e}")
        return

    with torch.no_grad():
        output = model(image_tensor)
        softmax = F.softmax(output, dim=1)
        confidence, predicted = torch.max(softmax, 1)
        class_idx = predicted.item()
        probs = softmax.squeeze().cpu().numpy()
        std_dev = np.std(probs)

        if std_dev < 0.1 or confidence.item() > 0.98:
            print(f"The provided image does not appear to be a lumbar spine MRI. Prediction not applicable.")
            return

        print(f"Model: {model_name}")
        print(f"Probabilities: {np.round(probs, 3)}")
        print(f"Predicted Class: {class_names[class_idx]}")


In [10]:
predict_with_model("/kaggle/input/mri-images/13.dcm", "Sagittal T1")


Model: Sagittal T1
Probabilities: [0.421 0.467 0.112]
Predicted Class: Mild


In [12]:
predict_with_model("/kaggle/input/mri-images/1.dcm", "Sagittal T2/STIR")


Model: Sagittal T2/STIR
Probabilities: [0.83  0.017 0.153]
Predicted Class: Normal


In [13]:
predict_with_model("/kaggle/input/mri-images/14.dcm", "Axial T2")


Model: Axial T2
Probabilities: [0.617 0.273 0.11 ]
Predicted Class: Normal


In [14]:
predict_with_model("/kaggle/input/nonmri-images/Brain.jpg", "Axial T2")


The provided image does not appear to be a lumbar spine MRI. Prediction not applicable.


In [15]:
predict_with_model("/kaggle/input/nonmri-images/Lungs.jpg", "Sagittal T2/STIR")


The provided image does not appear to be a lumbar spine MRI. Prediction not applicable.


In [16]:
predict_with_model("/kaggle/input/nonmri-images/Cat.jpg", "Axial T2")


The provided image does not appear to be a lumbar spine MRI. Prediction not applicable.
